In [1]:
import sys
import json
sys.path.insert(19, 'litepose-pose-estimation/src')
import torch
from torchvision import transforms
import lp_config.lp_common_config as cc
from lp_model.lp_litepose import LitePose
from lp_inference.lp_inference import inference, assocEmbedding
from lp_utils.lp_image_processing import drawHeatmap, drawKeypoints, drawSkeleton
import cv2
from PIL import Image
from datetime import datetime
import numpy as np
import pandas as pd
import imutils
sys.path.insert(20, 'src/referee_gloves_detector')
from color_labeler import ColorLabeler
import matplotlib.pyplot as plt

In [ ]:
# Leggo il file di configurazione
with open('config/config.json') as f:
    config_data = json.load(f)

# Ottengo i percorsi dei file
file_path_big_arch = config_data['path_big_arch']
file_path_csv_keypoints_webcam = config_data['path_csv_keypoints_webcam']

In [ ]:
model = LitePose().to(cc.config["device"])
model.load_state_dict(torch.load(file_path_big_arch, map_location=cc.config["device"]))

In [ ]:
# Red Filtering

image = cv2.imread('src/referee_gloves_detector/bloisi_nao_red_hands.png')
# image = cv2.imread('src/referee_gloves_detector/person_with_red_boxing_gloves.png')
cv2.imshow("Original", image)
 
result = image.copy()
 
image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
 
lower1 = np.array([0, 100, 100])
upper1 = np.array([5, 255, 255])

lower2 = np.array([174,100,100])
upper2 = np.array([179,255,255])
 
lower_mask = cv2.inRange(image, lower1, upper1)
upper_mask = cv2.inRange(image, lower2, upper2)
 
full_mask = lower_mask + upper_mask;
 
result = cv2.bitwise_and(result, result, mask=full_mask)

cv2.imshow('mask', full_mask)
cv2.imshow('result', result)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Red Filtering + Cropping

# image = cv2.imread('src/referee_gloves_detector/bloisi_nao_red_hands.png')
# image = cv2.imread('src/referee_gloves_detector/person_with_red_boxing_gloves.png')
image = cv2.imread('src/referee_gloves_detector/refeere_scaled.png')
cv2.imshow("Original", image)
 
result = image.copy()
 
image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

lower1 = np.array([0, 100, 100])
upper1 = np.array([5, 255, 255])

lower2 = np.array([174,100,100])
upper2 = np.array([179,255,255])
 
lower_mask = cv2.inRange(image, lower1, upper1)
upper_mask = cv2.inRange(image, lower2, upper2)
 
full_mask = lower_mask + upper_mask;
 
result = cv2.bitwise_and(result, result, mask=full_mask)

# #########################################
# codice crop full height

# creo una ROI a partire dalla maschera 
x,y,w,h = cv2.boundingRect(full_mask) 
rectangle_image = image.copy()
extra_margin = 25 # margine extra a sinistra e destra
w = w + extra_margin # sommo il margine extra alla larghezza della ROI
y = 0 # no cropping su altezza
h = image.shape[0] # no cropping su altezza
# creo un rettangolo a partire dalla ROI
cv2.rectangle(rectangle_image, (x-w,y), (x+(2*w),y+h), (255,0,0), 0)
# mostro immagine con rettangolo # cv2.imshow('rectangled', rectangle_image)
# croppo l'immagine usando la ROI
cropped_rectangle = rectangle_image[y:y+h, (x-w):x+(2*w)]
# riconverto l'immagine in colori umani
cropped = cv2.cvtColor(cropped_rectangle, cv2.COLOR_HSV2BGR)


cv2.imshow('mask', full_mask)
cv2.imshow('result', result)
cv2.imshow('cropped', cropped)
# ##########################################
 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# uri = "C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\example_shapes.png"
# uri = "C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\example_shapes_1.png"
# uri = "C:\\Users\\stolf\\dev\\Progetto Visione e Percezione\\src\\referee_gloves_detector\\example_shapes_2.png"
uri = "src/referee_gloves_detector/bloisi_nao_red_hands.png"

In [ ]:
# Apro la webcam e salvo al volo i frame e il timestamp

webcam=cv2.VideoCapture(0) 

if not webcam.isOpened():
    raise Exception("Errore nell'apertura della webcam")

keypoints_vec = []
timestamps = []
resize = transforms.Resize([224, 224])  
to_tensor = transforms.ToTensor()
ret,frame=webcam.read()
while ret:
    
    
    istante_attuale = datetime.now()
    stringa_istante = istante_attuale.strftime("%d/%m/%Y %H:%M:%S.%f")
    timestamps.append(stringa_istante) 
    
    frame = cv2.flip(frame, 1)
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = img[:,80:]
    img = img[:,:-80]
    

    im_pil = Image.fromarray(img)
    frame = resize(im_pil)
    tensor = to_tensor(frame)
    tensor = tensor.unsqueeze(0)
    
    
    if ret==True:
        output, keypoints = inference(model, tensor)
        keypoints_vec.append(keypoints)
        embedding = assocEmbedding(keypoints)
        frame_modified = drawSkeleton(tensor[0], embedding[0])
        #frame = drawKeypoints(tensor[0], keypoints[0])
        cv2.imshow("Pose estimation", frame_modified)
        key=cv2.waitKey(1) & 0xFF
        if key==ord("q"):
            break         
    ret, frame = webcam.read()
webcam.release()
cv2.destroyAllWindows()
print("Dimensioni dell'array di frame:", len(keypoints_vec))
print("Dimensioni dell'array di timestamp:", len(timestamps))

In [ ]:
# Webcam + cropping
# Apro la webcam e salvo al volo i frame e il timestamp

webcam=cv2.VideoCapture(0) 

if not webcam.isOpened():
    raise Exception("Errore nell'apertura della webcam")

keypoints_vec = []
timestamps = []
resize = transforms.Resize([224, 224])  
to_tensor = transforms.ToTensor()
ret,frame=webcam.read()
while ret:
    
    
    istante_attuale = datetime.now()
    stringa_istante = istante_attuale.strftime("%d/%m/%Y %H:%M:%S.%f")
    timestamps.append(stringa_istante) 
    
    frame = cv2.flip(frame, 1)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

lower1 = np.array([0, 100, 100])
upper1 = np.array([5, 255, 255])

lower2 = np.array([174,100,100])
upper2 = np.array([179,255,255])
 
lower_mask = cv2.inRange(image, lower1, upper1)
upper_mask = cv2.inRange(image, lower2, upper2)
 
full_mask = lower_mask + upper_mask;
 
result = cv2.bitwise_and(result, result, mask=full_mask)

# #########################################
# codice crop full height

# creo una ROI a partire dalla maschera 
x,y,w,h = cv2.boundingRect(full_mask) 
rectangle_image = image.copy()
extra_margin = 20 # margine extra a sinistra e destra
w = w + extra_margin # sommo il margine extra alla larghezza della ROI
y = 0 # no cropping su altezza
h = image.shape[0] # no cropping su altezza
# creo un rettangolo a partire dalla ROI
cv2.rectangle(rectangle_image, (x,y), (x+w,y+h), (255,0,0), 0)
# mostro immagine con rettangolo
cv2.imshow('rectangled', rectangle_image)
# croppo l'immagine usando la ROI
cropped_rectangle = rectangle_image[y:y+h, x:x+w]
# riconverto l'immagine in colori umani
cropped = cv2.cvtColor(cropped_rectangle, cv2.COLOR_HSV2BGR)


    cv2.imshow('mask', full_mask)
    cv2.imshow('cropped', cropped)
    
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = img[:,80:]
    img = img[:,:-80]
    

    im_pil = Image.fromarray(img)
    frame = resize(im_pil)
    tensor = to_tensor(frame)
    tensor = tensor.unsqueeze(0)
    
    
    if ret==True:
        output, keypoints = inference(model, tensor)
        keypoints_vec.append(keypoints)
        embedding = assocEmbedding(keypoints)
        frame_modified = drawSkeleton(tensor[0], embedding[0])
        #frame = drawKeypoints(tensor[0], keypoints[0])
        cv2.imshow("Pose estimation", frame_modified)
        key=cv2.waitKey(1) & 0xFF
        if key==ord("q"):
            break         
    ret, frame = webcam.read()
webcam.release()
cv2.destroyAllWindows()
print("Dimensioni dell'array di frame:", len(keypoints_vec))
print("Dimensioni dell'array di timestamp:", len(timestamps))

In [ ]:
# Salvo solo i 9 keypoints scelti per ogni frame

d = {'timestamp':[],'x': [], 'y': [], 'tag':[]}
df_keypoints = pd.DataFrame(data=d)
for timestamp, restricted_keypoints in zip(timestamps, keypoints_vec):
    for i in range(len(restricted_keypoints[0])):
        if not (restricted_keypoints[0][i]):
            df_temp = pd.DataFrame([{'timestamp': '01/01/1970 01:00:00','x':np.nan, 'y':np.nan, 'tag':np.nan}])
            df_keypoints = pd.concat([df_keypoints, df_temp])
        else:
            df_temp = pd.DataFrame(restricted_keypoints[0][i])
            df_temp['timestamp'] = timestamp
            df_keypoints = pd.concat([df_keypoints, df_temp])
df_keypoints.to_csv(file_path_csv_keypoints_webcam, index=False)
df_multiindex = df_keypoints.set_index(['timestamp'])
display(df_multiindex)